<a href="https://colab.research.google.com/github/SaburGH/DL_Python/blob/master/RecAudio010525_colabOnly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#!pip install pipwin

In [14]:
#!pipwin install pyaudio

In [15]:
#!pip install sounddevice

## For colab JavaScript is needed

In [16]:

from IPython.display import Audio, display, Javascript
from google.colab import output
import io
import base64
import soundfile as sf

# JavaScript to record audio
RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = () => resolve(reader.result)
  reader.readAsDataURL(blob)
})

var record = async function() {
  const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
  const recorder = new MediaRecorder(stream);
  const data = [];

  recorder.ondataavailable = event => data.push(event.data);
  recorder.start();

  await sleep(5000); // 5 seconds recording
  recorder.stop();

  await new Promise(resolve => recorder.onstop = resolve);

  const blob = new Blob(data, { type: 'audio/wav' });
  const base64data = await b2text(blob);
  return base64data;
};
"""

# Run JavaScript to record audio
display(Javascript(RECORD))
data = output.eval_js('record()')

# Decode and save the audio
header, encoded = data.split(',', 1)
audio_bytes = base64.b64decode(encoded)

with open("input.wav", "wb") as f:
    f.write(audio_bytes)

# Play the raw audio
display(Audio("input.wav"))

<IPython.core.display.Javascript object>

In [17]:
#!pip install noisereduce

In [18]:
import noisereduce as nr
import librosa
import matplotlib as plt

In [19]:

# Load the audio
y, sr = librosa.load("input.wav", sr=None)

# Use first 0.5 sec as noise sample
noise_clip = y[:int(sr*0.5)]
reduced = nr.reduce_noise(y=y, y_noise=noise_clip, sr=sr)

# Save and play cleaned audio
sf.write("cleaned.wav", reduced, sr)
display(Audio("cleaned.wav"))

<ipython-input-19-db9e0424dbb2>:2: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load("input.wav", sr=None)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
